<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
from xmlrpc.server import SimpleXMLRPCServer, SimpleXMLRPCRequestHandler
import xmlrpc.client
import datetime
import pickle
import numpy as np
import socket

TIMEOUT = 20
socket.setdefaulttimeout(TIMEOUT)

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

proxy_server = SimpleXMLRPCServer(("localhost", 8009), requestHandler=RequestHandler)
proxy_server.register_introspection_functions()

server_endpoint = xmlrpc.client.ServerProxy("http://localhost:8008")
stats_server = xmlrpc.client.ServerProxy("http://localhost:8018")

def log_event(event, duration):
    """Логирование события через сервер статистики"""
    try:
        stats_server.add_log(event, datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), duration)
    except Exception:
        print("Stats server unavailable. Logging skipped.")

def proxy_function(func_name, *args):
    """Выполнение функции на сервере с логированием времени выполнения"""
    start_time = datetime.datetime.now()
    try:
        # Подготовка аргументов для передачи
        new_args = [xmlrpc.client.Binary(pickle.dumps(arg)) if isinstance(arg, np.ndarray) else arg for arg in args]
        
        # Вызов функции на сервере
        result = getattr(server_endpoint, func_name)(*new_args)
        
#         # Десериализация результата
#         if isinstance(result, xmlrpc.client.Binary):
#             result = pickle.loads(result.data)
#             result 
        
        duration = (datetime.datetime.now() - start_time).total_seconds()
        log_event(func_name, duration)
        return result
    except Exception as e:
        print(f"Error in proxy function {func_name}: {e}")
        return f"Error: {e}"

# Регистрация функций
proxy_server.register_function(lambda *args: proxy_function('ping', *args), 'ping')
proxy_server.register_function(lambda *args: proxy_function('now', *args), 'now')
proxy_server.register_function(lambda *args: proxy_function('type', *args), 'type')
proxy_server.register_function(lambda *args: proxy_function('sum', *args), 'sum')
proxy_server.register_function(lambda *args: proxy_function('pow', *args), 'pow')
proxy_server.register_function(lambda *args: proxy_function('black_list_check', *args), 'black_list_check')
proxy_server.register_function(lambda *args: proxy_function('black_list_check_fio_birth', *args), 'black_list_check_fio_birth')
proxy_server.register_function(lambda *args: proxy_function('send_back_binary', *args), 'send_back_binary')
proxy_server.register_function(lambda *args: proxy_function('color_inversion', *args), 'color_inversion')
proxy_server.register_function(lambda *args: proxy_function('binary_threshold', *args), 'binary_threshold')
proxy_server.register_function(lambda *args: proxy_function('vertical_flip', *args), 'vertical_flip')

print("Proxy server listening on port 8009...")
proxy_server.serve_forever()


Proxy server listening on port 8009...
Stats server unavailable. Logging skipped.


127.0.0.1 - - [26/Oct/2024 22:08:38] "POST /RPC2 HTTP/1.1" 200 -


Stats server unavailable. Logging skipped.


127.0.0.1 - - [26/Oct/2024 22:09:04] "POST /RPC2 HTTP/1.1" 200 -


Stats server unavailable. Logging skipped.


127.0.0.1 - - [26/Oct/2024 22:09:30] "POST /RPC2 HTTP/1.1" 200 -


Stats server unavailable. Logging skipped.


127.0.0.1 - - [26/Oct/2024 22:09:38] "POST /RPC2 HTTP/1.1" 200 -


Stats server unavailable. Logging skipped.


127.0.0.1 - - [26/Oct/2024 22:09:47] "POST /RPC2 HTTP/1.1" 200 -
